In [1]:
from flask import Flask, request, jsonify
import json
from time import time
from textwrap import dedent
from uuid import uuid4

In [2]:
import hashlib
import json
from time import time

In [ ]:
# transactions > block > (pow) > chain => 모든 node가 동일한 chain을 공유

In [ ]:
class BlockChain(object):
    
    def __init__(self):
        self.chain = []
        self.current_transaction = []
        self.nodes = set()
        
        self.new_block(previouse_hash=1, proof=100)  # genesis block
        
    def new_transaction(self, sender, recipient, amount):
        
        self.current_transaction.append(
            {
                'sender':sender,
                'recipient':recipient,
                'amount':amount
            }
        )
        
        return self.last_block['index'] + 1
    
    def new_block(self, previous_hash=None, proof): # 거래내역, 이전블록헤쉬, proof
        
        block = {
            'index':len(self.chain) + 1,
            'timestamp': self.current_transaction,
            'transactions' : self.current_transaction,
            'proof': proof,
            'previous_hash': previous_hash or self.hash(self.chain[-1])
        }
        
        self.current_transaction = []
        
        self.chain.append(block)
        
        return block
    
    def register_node(self, address):
        parsed_url = urlparse(address)
        self.nodes.add(parsed_url.netloc)
        
    def valid_chain(self, chain):  # 현재 블록에 이전블록해쉬값 vs hash(이전블록)
        last_block = chain[0]
        current_index =1
        
        while current_index < len(chain):
            block = chain[current_index]
            
            if block['previous_hash'] != self.hash(last_block):
                return False
            last_block = block
            current_index += 1
            
        return True
    
    def resolve_conflicts(self):
        neighbors = self.nodes
        new_chain = None
        
        max_length = len(self.chain)
        
        for node in neighbors:
            tmp_url = 'http://' + str(node) + '/chain'
            response = requests.get(tem_url)
            if response.status_code = 200:
                length = response.json()['length']
                chain = response.json()['chain']
                
                if length > max_length and self.valid_chain(chain):
                    max_length = length
                    new_chain=chain
                    
            if new_chain:
                self.chain = new_chain
                return True
            
            return False
        
    def hash(block):
        block_string = json.dumps(block, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()
    
    def last_block(self):
        return self.chain[-1]
    
    def pow(self, last_proof):
        
        proof = 0
        
        while self.valid_proof(last_proof, proof) is False:
            proof += 1
            
        return proof
    
    def valid_proof(last_proof, proof):
        guess = str(last_proof + proof).encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == '0000'